In [1]:
import numpy as np
import pandas as pd
from itertools import product
import os
from Bio import SeqIO

In [10]:
import csv

In [2]:
nucleotides = ['A', 'C', 'G', 'T']
combinations_of_3 = list(product(nucleotides, repeat=3))

kolone = [''.join(combination) for combination in combinations_of_3]

In [6]:
nepoznate_kolone = []
nepoznate_kolone += ['Y', 'R', 'S', 'W', 'M', 'K', 'H', 'D', 'B', 'V', 'N']

In [3]:
# Sa greskom
genoms = ['NC_008563','NC_004431','NC_002655','NC_017632','NC_017641']

In [13]:
genoms = ['NC_017626', 'NC_017627', 'NC_013364', 'NC_013365', 'NC_013366', 'NC_013361', 'NC_013369', 
          'NC_013353', 'NC_013354', 'NC_018650', 'NC_018654', 'NC_018661', 'NC_018662', 'NC_018658', 
          'NC_018659', 'NC_018666', 'NC_008253', 'NC_011748', 'NC_017631', 'NC_008563', 'NC_009837', 
          'NC_009838', 'NC_020163', 'NC_010468', 'NC_012892', 'NC_012971', 'NC_013941', 'NC_017646', 
          'NC_004431', 'NC_017625', 'NC_017638', 'NC_011601', 'NC_009786', 'NC_009788', 'NC_009790', 
          'NC_009801', 'NC_011353', 'NC_011745', 'NC_002655', 'NC_007414', 'NC_017633', 'NC_012947', 
          'NC_009800', 'NC_011741', 'NC_011750', 'NC_017628', 'NC_007779', 'NC_010473', 'NC_012759', 
          'NC_020518', 'NC_000913', 'NC_016902', 'NC_016904', 'NC_017660', 'NC_011993', 'NC_022364', 
          'NC_017644', 'NC_017634', 'NC_017659', 'NC_017663', 'NC_022370', 'NC_012967', 'NC_017656', 
          'NC_017657', 'NC_011742', 'NC_011747', 'NC_011415', 'NC_011419', 'NC_013654', 'NC_013655', 
          'NC_010488', 'NC_010498', 'NC_002128', 'NC_002695', 'NC_013008', 'NC_013010', 'NC_017630', 
          'NC_017632', 'NC_011739', 'NC_011749', 'NC_011751', 'NC_017639', 'NC_017641', 'NC_017642', 
          'NC_017645', 'NC_007941', 'NC_007946', 'NC_017635', 'NC_017637', 'NC_017664', 'NC_017665', 
          'NC_017906', 'NC_017907', 'NC_017652', 'NC_017651']

In [14]:
len(genoms)

95

# Brojanje kodona za PAI ostrva

In [5]:
for genom in genoms:

    fasta_file = f"{genom}/{genom}.fasta"
    sequences = SeqIO.parse(fasta_file, "fasta")

    if os.path.exists(f"{genom}/patogena_ostrva.csv"):
        pai_ostrva = pd.read_csv(f"{genom}/patogena_ostrva.csv")

        niska = None
        for record in sequences:
            niska = record.seq

        for index, row in pai_ostrva.iterrows():
            i = row['Start']
            while row['End'] != i + 2:
                kodon = niska[i:i+3]
                if kodon in pai_ostrva.columns:
                    pai_ostrva.at[index, kodon] += 1
                else:
                    print(f"Kodon {kodon} nije validan.")
                    break
                i += 1
        pai_ostrva.to_csv(f"{genom}/patogena_ostrva.csv", index=False)
        print(f"Zavrsen {genom}.")
    else:
        print("Nema PAI ostrva.")


In [ ]:
konacna_tabela = pd.read_csv("konacna_tabela.csv")
print(konacna_tabela)
for genom in genoms:
        if os.path.exists(f"{genom}/patogena_ostrva.csv"):
                pai_ostrva = pd.read_csv(f"{genom}/patogena_ostrva.csv")
                for index, row in pai_ostrva.iterrows():
                        novi_red = {'ID': genom}
                        konacna_tabela = konacna_tabela.append(novi_red, ignore_index=True)
                        novi_red_indeks = konacna_tabela.index[-1]
                        if row['Type'] == 0:
                                konacna_tabela.at[novi_red_indeks,"Tip"] = "P" 
                        elif row['Type'] == 1:
                                 konacna_tabela.at[novi_red_indeks,"Tip"] = "C"
                        else:
                                konacna_tabela.at[novi_red_indeks,"Tip"] = "N"

                        for kodon in kolone:
                                if kodon in konacna_tabela.columns:
                                        konacna_tabela.at[novi_red_indeks,kodon] = pai_ostrva.at[index,kodon]
                                else:
                                        print(f"Kodon {kodon} nije validan.")
                konacna_tabela.to_csv(f"{genom}/konacna_tabela.csv", index=False)
                print(f"Zavrsen {genom}.")
        else:
                print("Nema PAI ostrva.")

In [ ]:
for genom in genoms:
    nepoznati_kodoni_pai_ostrva = []
    fasta_file = f"{genom}/{genom}.fasta"
    sequences = SeqIO.parse(fasta_file, "fasta")
    
    if os.path.exists(f"{genom}/patogena_ostrva.csv"):
        pai_ostrva = pd.read_csv(f"{genom}/patogena_ostrva.csv")

        niska = None
        for record in sequences:
            niska = record.seq
        
        for index, row in pai_ostrva.iterrows():
            nepoznati_kodoni = []
            i = row['Start']
            while row['End'] != i + 2:
                kodon = niska[i:i+3]
                if kodon not in pai_ostrva.columns:
                    print(f"Kodon {kodon} nije validan.")
                    for slovo in nepoznate_kolone:
                        kodon = kodon.replace(slovo, 'N')
                    nepoznati_kodoni.append(str(kodon))
                i += 1
            nepoznati_kodoni_pai_ostrva.append(nepoznati_kodoni)
        print(nepoznati_kodoni_pai_ostrva)
        with open(f"{genom}/nepoznati_kodoni_pai_ostrva.csv", "w") as f:
            wr = csv.writer(f)
            wr.writerows(nepoznati_kodoni_pai_ostrva)
        print(f"Zavrsen {genom}.")

[[], [], [], [], [], [], [], [], [], [], [], []]
Zavrsen NC_017626.
[[]]
Zavrsen NC_017627.
[[], [], [], [], [], [], [], [], [], []]
Zavrsen NC_013364.
[[]]
Zavrsen NC_013365.
[[], []]
Zavrsen NC_013366.
[[], [], [], [], [], [], [], []]
Zavrsen NC_013361.
[[]]
Zavrsen NC_013369.
[[], [], [], [], [], [], [], [], [], []]
Zavrsen NC_013353.
[[]]
Zavrsen NC_013354.
[[], [], [], [], [], [], [], [], [], [], [], [], []]
Zavrsen NC_018650.
[[]]
Zavrsen NC_018654.
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
Zavrsen NC_018661.
[[]]
Zavrsen NC_018662.
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]
Zavrsen NC_018658.
[[]]
Zavrsen NC_018659.
[[]]
Zavrsen NC_018666.
[[], [], [], [], [], [], [], [], [], []]
Zavrsen NC_008253.
[[], [], [], [], [], [], [], [], [], [], []]
Zavrsen NC_011748.
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Zavrsen NC_017631.
Kodon CAY nije validan.
Kodon AYT nije validan.
Kodon YTG nije validan.
Kodon TAW nije validan.
Kodon AWC nije v